In [15]:
# ==============================================================================
# ЯЧЕЙКА 1: УСТАНОВКА, ПОДКЛЮЧЕНИЕ К GDRIVE И КОНФИГУРАЦИЯ
# ==============================================================================

# Проверяем, что GPU доступен
#!nvidia-smi

# Устанавливаем lightkurve
!pip install lightkurve -q # -q значит "тихая" установка без лишнего вывода

# Импортируем все необходимые библиотеки
import numpy as np
import pandas as pd
import lightkurve as lk
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import glob
import os
import random

# --- КОНФИГУРАЦИЯ ПРОЕКТА ---
# /content/drive/MyDrive/NASA/data1/KEPLER/Light Curves
# Пути к вашим данным на Google Drive
GDRIVE_BASE_PATH = '/content/drive/MyDrive/NASA/data1/KEPLER/'
LC_FILES_PATH = GDRIVE_BASE_PATH + "Light Curves/"
KOI_CATALOG_PATH = GDRIVE_BASE_PATH + 'KOI (Cumulative List).csv'

# Параметры модели и данных
GLOBAL_VIEW_SIZE = 2048 # Длина вектора для глобального обзора (всей кривой)
LOCAL_VIEW_SIZE = 256    # Длина вектора для локального обзора ("свернутой" кривой)
BATCH_SIZE = 32          # Сколько примеров обрабатывать за раз. Если будет ошибка памяти, уменьшите до 32.
EPOCHS = 13              # Сколько раз прогонять все данные через модель

# Подключаем Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\nНастройка завершена!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Настройка завершена!


In [16]:
# ==============================================================================
# ЯЧЕЙКА 2: ЗАГРУЗКА КАТАЛОГА И ПОДГОТОВКА СПИСКОВ ФАЙЛОВ (с диагностикой)
# ==============================================================================

print("Загружаем каталог KOI...")
try:
    koi_df = pd.read_csv(KOI_CATALOG_PATH, comment='#')
    koi_df.columns = koi_df.columns.str.strip()
    print(f"Каталог KOI успешно загружен. Всего записей: {len(koi_df)}")

    confirmed_planets = koi_df[koi_df['koi_disposition'] == 'CONFIRMED']
    false_positives = koi_df[koi_df['koi_disposition'] == 'FALSE POSITIVE']

    positive_ids = confirmed_planets['kepid'].unique()
    negative_ids = false_positives['kepid'].unique()

    print(f"Найдено {len(positive_ids)} звезд с подтвержденными планетами в каталоге.")
    print(f"Найдено {len(negative_ids)} звезд с ложными срабатываниями в каталоге.")

    print("\nСканируем папку с кривыми блеска...")
    all_fits_files = glob.glob(LC_FILES_PATH + '*.fits')
    if not all_fits_files:
        print(LC_FILES_PATH)
        raise FileNotFoundError("Не найдено ни одного .fits файла! Проверьте путь LC_FILES_PATH.")

    file_map = {int(os.path.basename(f).split('-')[0].replace('kplr', '')): f for f in all_fits_files}
    print(f"Всего найдено {len(file_map)} уникальных файлов звезд.")

    # --- Создаем финальные списки данных для обучения ---
    positive_samples = []
    for kepid in positive_ids:
        if kepid in file_map:
            planet_params = confirmed_planets[confirmed_planets['kepid'] == kepid][['koi_period', 'koi_time0bk']].to_dict('records')
            positive_samples.append({'kepid': kepid, 'path': file_map[kepid], 'params': planet_params, 'label': 1})

    negative_samples = []
    for kepid in negative_ids:
        if kepid in file_map:
            negative_samples.append({'kepid': kepid, 'path': file_map[kepid], 'params': [], 'label': 0})

    # --- ДИАГНОСТИКА ---
    print("\n--- ДИАГНОСТИКА ---")
    print(f"Найдено совпадений (звезды, которые есть и в каталоге, и в файлах):")
    print(f"Положительных примеров (планеты): {len(positive_samples)}")
    print(f"Отрицательных примеров (не планеты): {len(negative_samples)}")
    print("--------------------")

    # Сбалансируем классы
    max_negatives = len(positive_samples) * 2
    if len(negative_samples) > max_negatives:
        negative_samples = random.sample(negative_samples, max_negatives)

    all_samples = positive_samples + negative_samples
    if len(all_samples) == 0:
        raise ValueError("После сопоставления с файлами не осталось ни одного примера для обучения. Проверьте содержимое архива.")

    random.shuffle(all_samples)
    print(f"\nВсего примеров для обучения после балансировки: {len(all_samples)}")

    # Делим на обучающую и валидационную выборки
    train_samples, val_samples = train_test_split(all_samples, test_size=0.2, random_state=42)

    # --- ЕЩЕ ДИАГНОСТИКА ---
    print(f"\nРазмер обучающей выборки (train_samples): {len(train_samples)}")
    print(f"Размер валидационной выборки (val_samples): {len(val_samples)}")
    print(f"Размер батча (BATCH_SIZE): {BATCH_SIZE}")

    if len(train_samples) < BATCH_SIZE:
        print("\nВНИМАНИЕ! Размер обучающей выборки меньше размера батча. Это вызовет ошибку 'length 0'.")
        print("Рекомендация: Скачайте больше данных или уменьшите BATCH_SIZE.")

except Exception as e:
    print(f"\nПроизошла ошибка: {e}")

Загружаем каталог KOI...
Каталог KOI успешно загружен. Всего записей: 9564
Найдено 1973 звезд с подтвержденными планетами в каталоге.
Найдено 4713 звезд с ложными срабатываниями в каталоге.

Сканируем папку с кривыми блеска...
Всего найдено 17156 уникальных файлов звезд.

--- ДИАГНОСТИКА ---
Найдено совпадений (звезды, которые есть и в каталоге, и в файлах):
Положительных примеров (планеты): 177
Отрицательных примеров (не планеты): 510
--------------------

Всего примеров для обучения после балансировки: 531

Размер обучающей выборки (train_samples): 424
Размер валидационной выборки (val_samples): 107
Размер батча (BATCH_SIZE): 32


In [17]:
# ==============================================================================
# ЯЧЕЙКА 3: DATA GENERATOR
# ==============================================================================

class DataGenerator(Sequence):
    """
    Генератор данных для Keras. "На лету" загружает и обрабатывает .fits файлы.
    """
    def __init__(self, samples, batch_size, global_size, local_size):
        self.samples = samples
        self.batch_size = batch_size
        self.global_size = global_size
        self.local_size = local_size

    def __len__(self):
        return int(np.floor(len(self.samples) / self.batch_size))

    def __getitem__(self, index):
        batch_samples = self.samples[index * self.batch_size:(index + 1) * self.batch_size]

        # Здесь мы будем хранить готовые данные для батча
        batch_global = np.zeros((self.batch_size, self.global_size, 1))
        batch_local = np.zeros((self.batch_size, self.local_size, 1))
        batch_y = np.zeros(self.batch_size)

        for i, sample in enumerate(batch_samples):
            try:
                # Читаем и обрабатываем кривую блеска
                # lc = lk.read(sample['path']).PDCSAP_FLUX.remove_nans()
                # Сначала читаем файл
                lc_file = lk.read(sample['path'])
                # Затем явно запрашиваем нужную кривую блеска
                lc = lc_file.get_lightcurve('PDCSAP_FLUX').remove_nans()
                flat_lc = lc.flatten(window_length=1001)

                # 1. Глобальный обзор
                # Нормализуем и интерполируем до нужного размера
                normalized_flux = flat_lc.flux.value / np.median(flat_lc.flux.value)
                global_view = np.interp(
                    np.linspace(0, len(normalized_flux) - 1, self.global_size),
                    np.arange(len(normalized_flux)),
                    normalized_flux
                )

                # 2. Локальный обзор
                if sample['label'] == 1: # Если это планета
                    # Сворачиваем по параметрам первой планеты в списке
                    period = sample['params'][0]['koi_period']
                    t0 = sample['params'][0]['koi_time0bk']
                    folded_lc = flat_lc.fold(period=period, epoch_time=t0)

                    # Интерполируем до нужного размера
                    folded_flux = folded_lc.flux.value
                    local_view = np.interp(
                        np.linspace(0, len(folded_flux) - 1, self.local_size),
                        np.arange(len(folded_flux)),
                        folded_flux
                    )
                else: # Если это не планета
                    # Просто берем случайный участок из глобального обзора
                    start = np.random.randint(0, self.global_size - self.local_size)
                    local_view = global_view[start:start + self.local_size]

                # Записываем в батч
                batch_global[i,] = (global_view - np.mean(global_view)) / np.std(global_view)
                batch_local[i,] = (local_view - np.mean(local_view)) / np.std(local_view)
                batch_y[i] = sample['label']

            except Exception as e:
                # Если файл битый или пустой, пропускаем его
                # print(f"Warning: Could not process file {sample['path']}. Error: {e}")
                pass # В реальном обучении лучше логировать ошибки

        return {'global_input': batch_global, 'local_input': batch_local}, batch_y

# Создаем экземпляры генераторов
train_gen = DataGenerator(train_samples, BATCH_SIZE, GLOBAL_VIEW_SIZE, LOCAL_VIEW_SIZE)
val_gen = DataGenerator(val_samples, BATCH_SIZE, GLOBAL_VIEW_SIZE, LOCAL_VIEW_SIZE)

print("\nГенераторы данных созданы.")


Генераторы данных созданы.


In [18]:
# ==============================================================================
# ЯЧЕЙКА 4: АРХИТЕКТУРА MULTI-BRANCH HYBRID NETWORK (MBHN)
# ==============================================================================

def build_mbhn(global_shape, local_shape):
    # --- Ветвь 1: Глобальный Охотник (ищет периодичность) ---
    global_input = Input(shape=global_shape, name='global_input')
    x = Conv1D(filters=16, kernel_size=7, activation='relu', padding='same')(global_input)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=5)(x)
    x = Conv1D(filters=32, kernel_size=5, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=5)(x)
    global_features = Flatten()(x)

    # --- Ветвь 2: Локальный Эксперт (анализирует форму транзита) ---
    local_input = Input(shape=local_shape, name='local_input')
    y = Conv1D(filters=16, kernel_size=7, activation='relu', padding='same')(local_input)
    y = BatchNormalization()(y)
    y = MaxPooling1D(pool_size=3)(y)
    y = Conv1D(filters=32, kernel_size=5, activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(pool_size=3)(y)
    y = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    local_features = Flatten()(y)

    # --- Слияние (Fusion) и Голова (Head) ---
    concatenated = Concatenate()([global_features, local_features])
    z = Dense(128, activation='relu')(concatenated)
    z = Dropout(0.5)(z)
    z = Dense(64, activation='relu')(z)
    output = Dense(1, activation='sigmoid')(z)

    # --- Создание и компиляция модели ---
    model = Model(inputs=[global_input, local_input], outputs=output)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Создаем модель
model = build_mbhn(global_shape=(GLOBAL_VIEW_SIZE, 1), local_shape=(LOCAL_VIEW_SIZE, 1))

# Выводим архитектуру
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ local_input         │ (None, 256, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 256, 16)   │        128 │ local_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_input        │ (None, 2048, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 16)   │         64 │ conv1d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 2048, 16)  │        128 │ global_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_10    │ (None, 85, 16)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2048, 16)  │         64 │ conv1d_10[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 85, 32)    │      2,592 │ max_pooling1d_10… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_8     │ (None, 409, 16)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 85, 32)    │        128 │ conv1d_13[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 409, 32)   │      2,592 │ max_pooling1d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_11    │ (None, 28, 32)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 409, 32)   │        128 │ conv1d_11[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 28, 64)    │      6,208 │ max_pooling1d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_9     │ (None, 81, 32)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 64)    │        256 │ conv1d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 2592)      │          0 │ max_pooling1d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 1792)      │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 4384)      │          0 │ flatten_4[0][0],  │
│ (Concatenate)       │                   │            │ flatten_5[0][0] 

 Total params: 581,889 (2.22 MB)

 Trainable params: 581,569 (2.22 MB)

 Non-trainable params: 320 (1.25 KB)

In [ ]:
# ==============================================================================
# ЯЧЕЙКА 5: ОБУЧЕНИЕ МОДЕЛИ (Исправленная версия)
# ==============================================================================

print("Начинаем обучение модели...")

# Добавим колбэки для сохранения лучшей модели и ранней остановки
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                       save_best_only=True,
                                       monitor='val_accuracy')
]

# ИСПРАВЛЕНИЕ: Мы убрали аргумент `workers=2`, так как он больше не поддерживается в `model.fit()`
# Keras автоматически будет использовать эффективную загрузку данных с генератором Sequence.
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\nОбучение завершено!")

# Визуализируем результаты
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.show()

Начинаем обучение модели...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/13
 4/13 ━━━━━━━━━━━━━━━━━━━━ 1:41 11s/step - accuracy: 1.0000 - loss: 0.6928